In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import random
from scipy.stats import pearsonr

Carrega a base de voos

In [2]:
international_flights = pd.read_csv('international_graph.csv')
international_flights = international_flights[['pais_origem', 'pais_destino', 'qtde_voos']]
column_list = international_flights.columns.values.tolist() 

source = international_flights['pais_origem'].unique()
target = international_flights['pais_destino'].unique()

Adequa os países que não tem registro de saída ou entrada

In [3]:
for country in source:
    if country not in target:  
        #print('pais_origem: '+str(country))
        df = pd.DataFrame([[country, country, 0]], columns=column_list)
        international_flights = international_flights.append(df, ignore_index = True)

for country in target: 
    if country not in source:
        #print('pais_destino: '+str(country))
        df = pd.DataFrame([[country, country, 0]], columns=column_list) 
        international_flights = international_flights.append(df, ignore_index = True) 

Constrói o grafo com a probabilidade de voos entre países

In [10]:
country_international_flights = international_flights.groupby(['pais_origem']).sum() 
transition_matrix = international_flights.copy()
transition_matrix = transition_matrix.drop(columns= ['qtde_voos'])

transition_matrix['prob'] = 0.

for index, row in international_flights.iterrows():
    weight = country_international_flights.loc[row['pais_origem']]
    weight = float(weight)
    if weight == 0:
        weight = 1
    transition_matrix.loc[index,'prob'] = row['qtde_voos']/weight

transition_matrix = transition_matrix.set_index('pais_origem')  
#transition_matrix.to_csv("transition_matrix_edge_list.csv")

'''Transforma o dataframe em uma matriz de correlacao'''
transition_matrix = transition_matrix.pivot(columns='pais_destino', values='prob') 
transition_matrix = transition_matrix.fillna(0)

print(transition_matrix.head())                  


pais_destino   Algeria  Argentina  Armenia     Aruba  Australia   Austria  \
pais_origem                                                                 
Algeria       0.000000   0.000074      0.0  0.000000   0.000000  0.009335   
Argentina     0.000294   0.000000      0.0  0.000042   0.002352  0.000042   
Armenia       0.000000   0.000000      0.0  0.000000   0.000000  0.024081   
Aruba         0.000000   0.000641      0.0  0.000000   0.000000  0.000000   
Australia     0.000000   0.000715      0.0  0.000000   0.000000  0.000423   

pais_destino  Azerbaijan  Bahamas   Bahrain  Bangladesh  ...   Ukraine  \
pais_origem                                              ...             
Algeria              0.0      0.0  0.000000    0.000000  ...  0.000000   
Argentina            0.0      0.0  0.000000    0.000000  ...  0.000000   
Armenia              0.0      0.0  0.000000    0.000000  ...  0.059569   
Aruba                0.0      0.0  0.000000    0.000000  ...  0.000000   
Australia       

Carrega o arquivo com o total diário de casos e pega o dia específico - 77

In [6]:
total_cases = pd.read_csv('total_cases_countries_normalized.csv')
total_cases = total_cases[['Name', 'Day', 'DailyCases']].set_index(['Name', 'Day'])

countries = transition_matrix.columns.to_list()

indexes = []
for index in total_cases.index:
    indexes.append(index)

daily_cases = []
for country in countries:
    if (country, 77) not in indexes:
        daily_cases.append(0.) 
    else:
        daily_cases.append(total_cases.loc[country, 77][0])

Cria a função pearsonr_pval para pegar o p value no grid search

In [7]:
def pearsonr_pval(x,y):
        return pearsonr(x,y)[1]

Cria o dataframe a ser utilizado e roda o gridsearch

In [17]:
df_final = pd.DataFrame()
df_final['daily_cases'] = daily_cases

contamination_rate = [float(x)/10.0 for x in range(10,31,1)]
initial_number = [x for x in range(10,510,10)] 

for v in initial_number:
    for r in contamination_rate:
        state_vector = [0]*129
        state_vector[24]=v  
        
        for i in range(78):
            if i == 0:
                state_vector = np.dot(state_vector, transition_matrix)
            else:
                state_vector = np.dot(state_vector, transition_matrix)
                state_vector_multiply = [state_country*r for state_country in state_vector]
                state_vector = state_vector_multiply
                
        df_final['state_vector'] = state_vector
        print("Initial Number: " + str(v) + " Contamination Rate: " + str(r))
        print(df_final.head())

        df_corr = df_final.corr(method='pearson', min_periods = 6)
        df_p = df_final.corr(method=pearsonr_pval, min_periods = 6)
        df_corr_display = df_corr.round(2).applymap(str) + ' p=' + df_p.round(2).applymap(str) + ''
        
        print("\n")
        print(df_corr_display)
        print("\n\n")
    print("------------------------------------------------------------------------------------------------------------------")

Initial Number: 10 Contamination Rate: 1.0
   daily_cases  state_vector
0         10.0      0.015754
1          9.0      0.029877
2         22.0      0.000979
3          0.0      0.001856
4         77.0      0.104358


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 10 Contamination Rate: 1.1
   daily_cases  state_vector
0         10.0     24.245994
1          9.0     45.980249
2         22.0      1.506326
3          0.0      2.855793
4         77.0    160.606978


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 10 Contamination Rate: 1.2
   daily_cases   state_vector
0         10.0   19695.144958
1          9.0   37349.991850
2         22.0    1223.596341
3          0.0    2319.775078
4         77.0  130461.870144


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0   

   daily_cases  state_vector
0         10.0  4.761490e+21
1          9.0  9.029719e+21
2         22.0  2.958162e+20
3          0.0  5.608279e+20
4         77.0  3.154041e+22


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 20 Contamination Rate: 2.1
   daily_cases  state_vector
0         10.0  2.038539e+23
1          9.0  3.865897e+23
2         22.0  1.266479e+22
3          0.0  2.401074e+22
4         77.0  1.350341e+24


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 20 Contamination Rate: 2.2
   daily_cases  state_vector
0         10.0  7.327902e+24
1          9.0  1.389668e+25
2         22.0  4.552591e+23
3          0.0  8.631104e+23
4         77.0  4.854048e+25


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 20 Contamination Ra

Initial Number: 30 Contamination Rate: 2.7
   daily_cases  state_vector
0         10.0  7.754152e+31
1          9.0  1.470502e+32
2         22.0  4.817407e+30
3          0.0  9.133159e+30
4         77.0  5.136399e+32


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 30 Contamination Rate: 2.8
   daily_cases  state_vector
0         10.0  1.275537e+33
1          9.0  2.418935e+33
2         22.0  7.924500e+31
3          0.0  1.502379e+32
4         77.0  8.449234e+33


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 30 Contamination Rate: 2.9
   daily_cases  state_vector
0         10.0  1.901824e+34
1          9.0  3.606631e+34
2         22.0  1.181542e+33
3          0.0  2.240046e+33
4         77.0  1.259780e+35


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p

   daily_cases  state_vector
0         10.0      0.078772
1          9.0      0.149384
2         22.0      0.004894
3          0.0      0.009278
4         77.0      0.521792


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 50 Contamination Rate: 1.1
   daily_cases  state_vector
0         10.0    121.229970
1          9.0    229.901247
2         22.0      7.531630
3          0.0     14.278964
4         77.0    803.034892


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 50 Contamination Rate: 1.2
   daily_cases   state_vector
0         10.0   98475.724788
1          9.0  186749.959252
2         22.0    6117.981703
3          0.0   11598.875392
4         77.0  652309.350719


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 50 Contaminat

Initial Number: 60 Contamination Rate: 1.8
   daily_cases  state_vector
0         10.0  4.280923e+18
1          9.0  8.118368e+18
2         22.0  2.659600e+17
3          0.0  5.042247e+17
4         77.0  2.835710e+19


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 60 Contamination Rate: 1.9
   daily_cases  state_vector
0         10.0  2.751578e+20
1          9.0  5.218108e+20
2         22.0  1.709467e+19
3          0.0  3.240921e+19
4         77.0  1.822662e+21


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 60 Contamination Rate: 2.0
   daily_cases  state_vector
0         10.0  1.428447e+22
1          9.0  2.708916e+22
2         22.0  8.874485e+20
3          0.0  1.682484e+21
4         77.0  9.462123e+22


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p

...

Initial Number: 440 Contamination Rate: 2.7
   daily_cases  state_vector
0         10.0  1.137276e+33
1          9.0  2.156736e+33
2         22.0  7.065530e+31
3          0.0  1.339530e+32
4         77.0  7.533385e+33


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 440 Contamination Rate: 2.8
   daily_cases  state_vector
0         10.0  1.870787e+34
1          9.0  3.547772e+34
2         22.0  1.162260e+33
3          0.0  2.203490e+33
4         77.0  1.239221e+35


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 440 Contamination Rate: 2.9
   daily_cases  state_vector
0         10.0  2.789342e+35
1          9.0  5.289725e+35
2         22.0  1.732929e+34
3          0.0  3.285401e+34
4         77.0  1.847677e+36


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.

Initial Number: 460 Contamination Rate: 1.5
   daily_cases  state_vector
0         10.0  2.625355e+13
1          9.0  4.978739e+13
2         22.0  1.631049e+12
3          0.0  3.092251e+12
4         77.0  1.739051e+14


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 460 Contamination Rate: 1.6
   daily_cases  state_vector
0         10.0  3.779199e+15
1          9.0  7.166895e+15
2         22.0  2.347895e+14
3          0.0  4.451296e+14
4         77.0  2.503365e+16


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 460 Contamination Rate: 1.7
   daily_cases  state_vector
0         10.0  4.024649e+17
1          9.0  7.632369e+17
2         22.0  2.500386e+16
3          0.0  4.740397e+16
4         77.0  2.665953e+18


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.

Initial Number: 470 Contamination Rate: 2.5
   daily_cases  state_vector
0         10.0  3.242513e+30
1          9.0  6.149122e+30
2         22.0  2.014470e+29
3          0.0  3.819165e+29
4         77.0  2.147861e+31


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 470 Contamination Rate: 2.6
   daily_cases  state_vector
0         10.0  6.644295e+31
1          9.0  1.260028e+32
2         22.0  4.127888e+30
3          0.0  7.825923e+30
4         77.0  4.401222e+32


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 470 Contamination Rate: 2.7
   daily_cases  state_vector
0         10.0  1.214817e+33
1          9.0  2.303787e+33
2         22.0  7.547271e+31
3          0.0  1.430862e+32
4         77.0  8.047025e+33


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.

             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 490 Contamination Rate: 1.4
   daily_cases  state_vector
0         10.0  1.378663e+11
1          9.0  2.614504e+11
2         22.0  8.565191e+09
3          0.0  1.623846e+10
4         77.0  9.132349e+11


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 490 Contamination Rate: 1.5
   daily_cases  state_vector
0         10.0  2.796574e+13
1          9.0  5.303439e+13
2         22.0  1.737422e+12
3          0.0  3.293919e+12
4         77.0  1.852468e+14


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 490 Contamination Rate: 1.6
   daily_cases  state_vector
0         10.0  4.025668e+15
1          9.0  7.634302e+15
2         22.0  2.501019e+14
3          0.0  4.741597e+14
4         77.0  2.66

             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 500 Contamination Rate: 2.3
   daily_cases  state_vector
0         10.0  5.615864e+27
1          9.0  1.064996e+28
2         22.0  3.488957e+26
3          0.0  6.614596e+26
4         77.0  3.719984e+28


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 500 Contamination Rate: 2.4
   daily_cases  state_vector
0         10.0  1.488123e+29
1          9.0  2.822086e+29
2         22.0  9.245233e+27
3          0.0  1.752772e+28
4         77.0  9.857420e+29


             daily_cases state_vector
daily_cases    1.0 p=1.0   0.57 p=0.0
state_vector  0.57 p=0.0    1.0 p=1.0



Initial Number: 500 Contamination Rate: 2.5
   daily_cases  state_vector
0         10.0  3.449482e+30
1          9.0  6.541619e+30
2         22.0  2.143053e+29
3          0.0  4.062942e+29
4         77.0  2.28